In [1]:
import sys
sys.path.append('/Users/gabesmithline/Desktop/SRG/melo_project/marketsim')
from egta.reductions.dpr import DPRGame
from egta.game import Game

import numpy as np
from collections import defaultdict
from egta.process_data import process_game_data
'''
test data
'''


'\ntest data\n'

## Test data

In [2]:
#example payoff matrix for 6 agents and 2 strategies
"""
Melo  Lit   
0  6   | 0 50
1  5   | 50 10
2  4   | 30 20
3  3   | 50 50
4  2   | 40 70
5  1   | 10 100
6  0   | 100 0 
"""

#example raw data input for each strategy profile for 6 agents and 2 strategies
raw_data_single = [[ #2, 4
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 30],
    ['agent_3', 'MELO + ZI STRAT', 40],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 20],
    ['agent_5', 'MELO + ZI STRAT', 60],
    ['agent_6', 'MELO + ZI STRAT', 45]
],
[ #2, 4
    ['agent_1', 'MELO + ZI STRAT', 45],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 20],
    ['agent_3', 'MELO + ZI STRAT', 13],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 2],
    ['agent_5', 'MELO + ZI STRAT', 90],
    ['agent_6', 'MELO + ZI STRAT', 10],
],
[ #3, 3
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 30],
    ['agent_3', 'MELO + ZI STRAT', 40],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 20],
    ['agent_5', 'LIT_ORDERBOOK + ZI STRAT', 60],
    ['agent_6', 'MELO + ZI STRAT', 45]
],
[ #4, 2
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 30],
    ['agent_3', 'MELO + ZI STRAT', 40],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 20],
    ['agent_5', 'LIT_ORDERBOOK + ZI STRAT', 60],
    ['agent_6', 'LIT_ORDERBOOK + ZI STRAT', 45]
],
[ #5, 1
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 30],
    ['agent_3', 'LIT_ORDERBOOK + ZI STRAT', 40],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 20],
    ['agent_5', 'LIT_ORDERBOOK + ZI STRAT', 60],
    ['agent_6', 'LIT_ORDERBOOK + ZI STRAT', 45]
],
[ #1, 5
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 30],
    ['agent_3', 'MELO + ZI STRAT', 40],
    ['agent_4', 'MELO + ZI STRAT', 20],
    ['agent_5', 'MELO + ZI STRAT', 60],
    ['agent_6', 'MELO + ZI STRAT', 45]
], 
[ #0, 6
    ['agent_1', 'MELO + ZI STRAT', 50],
    ['agent_2', 'MELO + ZI STRAT', 30],
    ['agent_3', 'MELO + ZI STRAT', 40],
    ['agent_4', 'MELO + ZI STRAT', 20],
    ['agent_5', 'MELO + ZI STRAT', 60],
    ['agent_6', 'MELO + ZI STRAT', 45]
]]


game_single = process_game_data(raw_data_single)
for row in game_single.get_payoff_matrix():
    print(row)

new_raw_data = [
[ #5, 1
    ['agent_1', 'MELO + ZI STRAT', 11],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 11],
    ['agent_3', 'LIT_ORDERBOOK + ZI STRAT', 11],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 11],
    ['agent_5', 'LIT_ORDERBOOK + ZI STRAT', 11],
    ['agent_6', 'LIT_ORDERBOOK + ZI STRAT', 3]
],
[ #3, 3
    ['agent_1', 'MELO + ZI STRAT', 9],
    ['agent_2', 'LIT_ORDERBOOK + ZI STRAT', 9],
    ['agent_3', 'MELO + ZI STRAT', 9],
    ['agent_4', 'LIT_ORDERBOOK + ZI STRAT', 9],
    ['agent_5', 'LIT_ORDERBOOK + ZI STRAT', 9],
    ['agent_6', 'MELO + ZI STRAT', 9]
]


]


prior_payoff_matrix, new_payoff_matrix = game_single.update_payoffs(new_raw_data)  
print("Prior payoff matrix")
for row in prior_payoff_matrix:
    print(row)
print("New payoff matrix")
for row in new_payoff_matrix:
    print(row)

    



(('LIT_ORDERBOOK + ZI STRAT', 2), ('MELO + ZI STRAT', 4))
Repeat profile: (('LIT_ORDERBOOK + ZI STRAT', 2), ('MELO + ZI STRAT', 4))
['# LIT_ORDERBOOK + ZI STRAT', '# MELO + ZI STRAT', 'Payoff (LIT_ORDERBOOK + ZI STRAT)', 'Payoff (MELO + ZI STRAT)']
[2, 4, 18.0, 44.125]
[3, 3, 36.666666666666664, 45.0]
[4, 2, 38.75, 45.0]
[5, 1, 39.0, 50.0]
[1, 5, 30.0, 43.0]
[0, 6, 0, 40.833333333333336]
Prior payoff matrix
['# LIT_ORDERBOOK + ZI STRAT', '# MELO + ZI STRAT', 'Payoff (LIT_ORDERBOOK + ZI STRAT)', 'Payoff (MELO + ZI STRAT)']
[2, 4, 18.0, 44.125]
[3, 3, 36.666666666666664, 45.0]
[4, 2, 38.75, 45.0]
[5, 1, 39.0, 50.0]
[1, 5, 30.0, 43.0]
[0, 6, 0, 40.833333333333336]
New payoff matrix
['# LIT_ORDERBOOK + ZI STRAT', '# MELO + ZI STRAT', 'Payoff (LIT_ORDERBOOK + ZI STRAT)', 'Payoff (MELO + ZI STRAT)']
[2, 4, 18.0, 44.125]
[3, 3, 22.833333333333332, 27.0]
[4, 2, 38.75, 45.0]
[5, 1, 24.2, 30.5]
[1, 5, 30.0, 43.0]
[0, 6, 0, 40.833333333333336]


In [3]:
dprgame = DPRGame(game_single, game_single.num_players, 2)



AttributeError: 'DPRGame' object has no attribute 'scaling_factor'

## Check Ratio Validator

In [4]:
dprgame.check_reduced_players_ratio()


True